# ReAct Agent with Query Engine (RAG) Tools

In [17]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core.settings import Settings
from llama_index.core import VectorStoreIndex
import os

##  Build the Index

In [18]:
# Azure OpenAI configuration
llm_deployment = os.getenv("AZURE_OPENAI_LLM_DEPLOYMENT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
model = "gpt-4o"
temperature = 0.7
max_tokens = 4000
embedding_model = "text-embedding-3-large"
embedding_deployment = embedding_model


llm_config = {
    "api_key": api_key,
    "api_version": api_version,
    "azure_endpoint": azure_endpoint,
    "deployment_name": llm_deployment,
    "model": model,
    "temperature": temperature,
    "max_tokens": int(max_tokens) if max_tokens is not None else None
}

embedding_config = {
    "api_key": api_key,
    "api_version": api_version,
    "azure_endpoint": azure_endpoint,
    "deployment_name": embedding_deployment,
    "model": embedding_model,
    "dimensions": 1024
}

# Set up LLM and embedding models
Settings.embed_model = AzureOpenAIEmbedding(**embedding_config)
Settings.llm = AzureOpenAI(**llm_config)

# (Optional) If you want to load your own Excel files, you can implement the following code.
# For example, for a specific Excel file:
# if not index_loaded:
# Load your data (e.g., replace with your actual files)
excel_files = [
    './data/BP_Excel.xlsx',
    './data/nvidia_quarterly_revenue_trend_by_market.xlsx',
    './data/titanic_train.csv',
    # Add more files as needed
]

docs = SimpleDirectoryReader(input_files=excel_files).load_data()

# Build index
index = VectorStoreIndex.from_documents(docs)

# Persist index
index.storage_context.persist(persist_dir="./data")


## Build the Query Engine

In [19]:
# Create a query engine
query_engine = index.as_query_engine(similarity_top_k=3)

# Setup Query Engine Tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="excel_query",
            description=(
                "Provides information based on the provided Excel files. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

## Setup ReAct Agent

In [20]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=Settings.llm,
    verbose=True
)


## Run Some Example Queries

In [21]:
response = agent.chat("What insights can you provide from the BP Excel file?")
print(str(response))

response = agent.chat("Analyze the quarterly revenue trends for Nvidia.")
print(str(response))

# You can continue to run queries based on the files you have in the 'data' directory.

> Running step 08732cff-a4c3-4650-ba67-248350e430b9. Step input: What insights can you provide from the BP Excel file?
Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: excel_query
Action Input: {'input': 'Please provide insights from the BP Excel file.'}
Observation: The BP Excel file provides a detailed overview of their financial and operational performance. Key insights include:

1. **Margin Share**: The margin share from convenience and electrification has shown a gradual increase, reaching 0.291 in the latest period.
2. **Sales and Revenues**: Total customers and products sales and other revenues have shown variability with significant figures for Castrol and other convenience and mobility, refining and trading, and petrochemicals.
3. **Average Realizations**: The average realizations for oil and gas have fluctuated, with notable changes in total liquids and natural gas prices over the quarters.
4. **Production Cost